# OpenAI Function Calling


**Notes**:
- LLMs don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.
- OpenAI has announced the release of an updated GPT-3.5-Turbo model and the deprecation of the ```gpt-3.5-turbo-0613``` model. The gpt-3.5-turbo-0613 model, which has been utilized in this Short Course since its launch in October 2023, will be replaced by the ```gpt-3.5-turbo``` model.

**Takeaway**:
- openai.ChatCompletion.create() is deprecated. Use client = OpenAI() and client.chat.completions.create() instead.
- you can force the model to call a function by using the `function_call` parameter.
- when taking a function call, the model will return a `response_message` object with empty content but non-empty `response_message.function_call.arguments`


In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    # json.dumps converts a Python object to a JSON-formatted string.
    return json.dumps(weather_info)

In [6]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location", # Description is counted as tokens
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA", # Description is counted as tokens
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [7]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [10]:
!pip install openai

In [15]:
from openai import OpenAI

> Note: This notebook was updated in June 2024. Consequently, we are now using the ```gpt-3.5-turbo model``` instead of the ```gpt-3.5-turbo-0613``` model featured by the instructor in the video lesson.

In [ ]:
# Call the ChatCompletion endpoint - OUTDATED
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions
# )

In [16]:
client = OpenAI()

In [18]:
response = client.chat.completions. create(
    model = "gpt-4.1",
    messages = messages,
    functions = functions
)

> Note: The following result may differ slightly from the one shown by the instructor in the video lesson due to the model being updated.

In [19]:
print(response)

ChatCompletion(id='chatcmpl-C1UQLtCl7xsi2OgAmjOr201BBXkY5', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA","unit":"fahrenheit"}', name='get_current_weather'), tool_calls=None))], created=1754471313, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_51e1070cf2', usage=CompletionUsage(completion_tokens=22, prompt_tokens=79, total_tokens=101, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [22]:
response_message = response.choices[0].message

In [24]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA","unit":"fahrenheit"}', name='get_current_weather'), tool_calls=None)

In [25]:
response_message.content

In [26]:
response_message.function_call

FunctionCall(arguments='{"location":"Boston, MA","unit":"fahrenheit"}', name='get_current_weather')

In [31]:
response_message.function_call.arguments

'{"location":"Boston, MA","unit":"fahrenheit"}'

In [33]:
# json.loads is a Python function that parses a JSON-formatted string and returns the corresponding Python object (e.g., dict)
json.loads(response_message.function_call.arguments)

{'location': 'Boston, MA', 'unit': 'fahrenheit'}

In [34]:
args = json.loads(response_message.function_call.arguments)

In [35]:
get_current_weather(args)

'{"location": {"location": "Boston, MA", "unit": "fahrenheit"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

* Pass a message that is not related to a function.

In [36]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

> Note: This notebook was updated in June 2024. Consequently, we are now using the ```gpt-3.5-turbo model``` instead of the ```gpt-3.5-turbo-0613``` model featured by the instructor in the video lesson.

In [ ]:
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions,
# )

In [37]:
response = client.chat.completions.create(
    model = "gpt-4.1",
    messages = messages,
    functions = functions
)

In [38]:
print(response)

ChatCompletion(id='chatcmpl-C1UVpoWJAtOR07XAN7P93TgF6fP95', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today? 😊', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754471653, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_799e4ca3f1', usage=CompletionUsage(completion_tokens=11, prompt_tokens=74, total_tokens=85, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


* Pass additional parameters to force the model to use or not a function.

In [40]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions,
#     function_call="auto",
# )

response = client.chat.completions.create(
    model = "gpt-4.1",
    messages = messages,
    functions = functions,
    function_call = 'auto'
)
print(response)

ChatCompletion(id='chatcmpl-C1UWvlSVls2mCe28hkTtUcfcE5t6q', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today? 😊', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754471721, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_51e1070cf2', usage=CompletionUsage(completion_tokens=11, prompt_tokens=74, total_tokens=85, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


* Use mode 'none' for function call.

In [42]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions,
#     function_call="none",
# )

response = client.chat.completions.create(
    model = "gpt-4.1",
    messages = messages,
    functions = functions,
    function_call = 'none'
)

print(response)

ChatCompletion(id='chatcmpl-C1UXlgOdBYwMcPN1RSijvqGK75rnX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today? 😊', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754471773, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_799e4ca3f1', usage=CompletionUsage(completion_tokens=10, prompt_tokens=75, total_tokens=85, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


* When the message should call a function and still uses mode 'none'.

In [43]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions,
#     function_call="none",
# )

response = client.chat.completions.create(
    model = "gpt-4.1",
    messages = messages,
    functions = functions,
    function_call = 'none'
)

print(response)

ChatCompletion(id='chatcmpl-C1UY4MlwjADhMj7MzQIAaCJT49IqF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Let me check the current weather in Boston for you.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754471792, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_b3f1157249', usage=CompletionUsage(completion_tokens=11, prompt_tokens=79, total_tokens=90, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


* Force calling a function.

In [44]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions,
#     function_call={"name": "get_current_weather"},
# )

response = client.chat.completions.create(
    model = "gpt-4.1",
    messages = messages,
    functions = functions,
    function_call={"name": "get_current_weather"}
)

# The model just hallucinates (come up with a random location)
print(response)

ChatCompletion(id='chatcmpl-C1UYRMqUpvgVq929HM8wRTJDkxAKU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"San Francisco, CA"}', name='get_current_weather'), tool_calls=None))], created=1754471815, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_b3f1157249', usage=CompletionUsage(completion_tokens=8, prompt_tokens=84, total_tokens=92, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


* Final notes.

In [46]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
# response = openai.ChatCompletion.create(
#     # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
#     model="gpt-3.5-turbo",
#     messages=messages,
#     functions=functions,
#     function_call={"name": "get_current_weather"},
# )

response = client.chat.completions.create(
    model = "gpt-4.1",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

ChatCompletion(id='chatcmpl-C1Ucim3gI5SnQ8a0ZKm9yw92iNyN9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA","unit":"fahrenheit"}', name='get_current_weather'), tool_calls=None))], created=1754472080, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_b3f1157249', usage=CompletionUsage(completion_tokens=12, prompt_tokens=89, total_tokens=101, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [49]:
# messages.append(response["choices"][0]["message"])
messages.append(response.choices[0].message)

In [55]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)
observation

'{"location": {"location": "Boston, MA", "unit": "fahrenheit"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [56]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [60]:
# response = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=messages,
# )

response = client.chat.completions.create(
    model = "gpt-4.1",
    messages = messages,
)
print(response.choices[0].message.content)

Right now in Boston, it's 72°F, sunny, and windy! If you’re heading outside, you might want to bring a light jacket just in case the wind picks up. Let me know if you need a more detailed forecast or info for another day!
